<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Calculate the Vapor Pressure Deficit for the UFS S2S </h1>
By: Kayla Besong, PhD
    <br>
Last Edited: 11/22/23
<br>
<br>    
<br>
Takes models/variables downloaded and calculates the vapor pressure defict. The vapor pressure deficit calculation is in File_concat_mod_functions.ipynb, as is the function that computes the 24HR AVG, MIN, MAX outputs. 
<div style="clear:both"></div>
</div>

<hr style="height:2px;">

## Import needed libraries, etc.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from dask.distributed import Client, LocalCluster
import dask.array as da
import os
import glob
from metpy.units import units
import math

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None

## OPTIONAL: Establish a dask client. This is a lot of data.

In [ ]:
Cluster = LocalCluster(n_workers = 8, threads_per_worker=4, memory_limit='30GB',  processes=True)
#Cluster = LocalCluster()

In [ ]:
client = Client(Cluster)
client

### The integral notebook of functions to run

In [3]:
%run File_concat_mod_functions.ipynb

the main function to run is:

    return_concat(model, variable, prototype = None)

Where the prototype is only needed for the UFS_S2S model choice. 

## UFS
expected timesteps = 1460

In [ ]:
output_dir = 'database_files'

In [ ]:
model = 'UFS_S2S'
prototypes = [5,6,7,8]

In [ ]:
var_list = [get_model_var(model,'TEMP'), get_model_var(model,'RH')]
var_list

In [ ]:
%%time

for p in prototypes:

    parent_dirs = glob.glob(os.path.join(f'{model}/{p}/', '*'))
    
    for i in parent_dirs:
                
        dir_maker(os.path.join(f'{output_dir}', f'{i}'))
        
        temp_files = glob.glob(os.path.join(f'{i}/{var_list[0]}', f'*{var_list[0]}*'))
        rh_files = glob.glob(os.path.join(f'{i}/{var_list[1]}', f'*{var_list[1]}*'))
    
        temp_file_cc = xr.open_mfdataset(temp_files, combine = 'nested', concat_dim = 'valid_time', chunks = get_chunk(model))
        rh_file_cc = xr.open_mfdataset(rh_files, combine = 'nested', concat_dim = 'valid_time', chunks = get_chunk(model))
        
        temp_file_cc = temp_file_cc.sortby('valid_time')
        rh_file_cc = rh_file_cc.sortby('valid_time')
        
        ufs_t2m = temp_file_cc.t2m - 273.15
        ufs_rh = rh_file_cc.r2/100.
    
        ufs_vpd = vapor_pressure_deficit(ufs_t2m, ufs_rh)
        
        resampler_UFS('vpd', ufs_vpd.chunk(get_chunk(model)), output_dir, i, p)